In [1]:
import json
import os
from langchain_core.tools import Tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# Custom output parser for JSON formatting
def parse_output(text):
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        return {
            "answer": "",
            "explanation": f"Output parsing error: Invalid JSON output received - {text}"
        }

# Sample function for the tool
def pdf_retriever_function(input_data):
    # This is a placeholder function
    return "Relevant data from the PDF document."

# Define tools
tools = [
    Tool(
        name="PDF Retriever",
        func=pdf_retriever_function,
        description="Retrieve relevant information from the uploaded PDF"
    )
]

# Define the prompt template with required variables
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

When providing the Final Answer, format it as a JSON object with the following structure:
{
    "answer": "concise response to the question",
    "explanation": "optional extra details if needed, else empty string"
}

Question: {input}

{agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

# Initialize the Ollama model
model = Ollama(model="deepseek-llm:7b-chat")

# Create the ReAct agent
agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt
)

# Create an agent executor
executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

# Function to load a PDF and create a vector store
def load_pdf_and_create_vector_store(pdf_path):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()
    
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(docs, embeddings)
    return vectorstore

# Function to query the agent and ensure JSON response
def query_agent(question):
    response = executor.invoke({"input": question})
    
    # Log the raw response for debugging
    print("Raw response from agent:", response)
    
    # Parse the response to ensure JSON format
    return parse_output(response['output'])

# Example usage
if __name__ == "__main__":
    # Path to the PDF file
    pdf_path = "English.pdf"  # Replace with your PDF path

    # Load the PDF and create the vector store
    vectorstore = load_pdf_and_create_vector_store(pdf_path)

    # Example query
    query = "What is the role of counselling in preventing drug addiction?"
    response_json = query_agent(query)

    # Print the structured JSON response
    print("Structured JSON response:", json.dumps(response_json, indent=2))

/var/folders/_x/8clrdsj55jv7dc605rcxqx3r0000gn/T/ipykernel_4931/2946585219.py:49: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model="deepseek-llm:7b-chat")


ValueError: Prompt missing required variables: {'tool_names', 'tools'}